In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")

print("Setup complete.")

Setup complete.


In [7]:
import numpy as np
from darth import fn
from darth.nn import Linear, Sequential
from darth.fn import av

In [8]:
fan_in, fan_out = 4, 2

seed = 21
np.random.seed(seed)
data = np.random.randn(4, fan_in)
print(f'data : \n{data}\n')

# l1 = Linear(fan_in, 3, activation=ReLU)
# l2 = Linear(3, 2)
# l3 = Linear(2, fan_out)

# o1 = l1.forward(data)
# o2 = l2.forward(o1)
# o3 = l3.forward(o2)

# s1 = Sequential(l1, l2, l3)
# out = s1.forward(data)

# print(f'out == o3 : {np.array_equal(out, o3)}')
# print(f'\nout : \n{out}')
# print(f'\no3 : \n{o3}')

data : 
[[-0.05196425 -0.11119605  1.0417968  -1.25673929]
 [ 0.74538768 -1.71105376 -0.20586438 -0.23457129]
 [ 1.12814404 -0.01262595 -0.61320029  1.3736885 ]
 [ 1.61099198 -0.68922827  0.69192371 -0.4481156 ]]



In [17]:
l1 = Linear(fan_in, 4, activation=av.ReLU)
l2 = Linear(4, fan_out, activation=av.ReLU)
# l1.add_module('l2', l2)
l1

[Linear] Patching Linear with activation: <class 'darth.fn.activation.ReLU'>
[Linear] activation of type: <class 'type'>
[Linear] Adding submodule: activation of type ReLU
[Linear] Patching Linear with activation: <class 'darth.fn.activation.ReLU'>
[Linear] activation of type: <class 'type'>
[Linear] Adding submodule: activation of type ReLU


Linear(
  fan_in = 4,
  fan_out = 4,
  activation = ReLU(),
)

In [23]:
o1 = l1.forward(data)
o2 = l2.forward(o1)
print(f'\no1 : \n{o1}')
print(f'\no2 : \n{o2}')
l1, l2

[Linear] forward called
[Linear] _forward Linear called with input shape (4, 4)
[Linear] apply_activation called with activation: None
[Linear] forward called
[Linear] _forward Linear called with input shape (4, 4)
[Linear] apply_activation called with activation: None

o1 : 
[[0.         0.         0.         0.        ]
 [0.02108339 0.         0.         0.        ]
 [0.         0.57684905 1.5010857  0.45935267]
 [0.         0.         0.75157488 0.        ]]

o2 : 
[[0.         0.        ]
 [0.03831062 0.        ]
 [0.43566042 0.        ]
 [0.3166436  0.12778457]]


(Linear(
   fan_in = 4,
   fan_out = 4,
   activation = ReLU(),
 ),
 Linear(
   fan_in = 4,
   fan_out = 2,
   activation = ReLU(),
 ))

In [25]:
s = Sequential(
    Linear(fan_in, 4, activation=av.ReLU),
    Linear(4, fan_out, activation=av.ReLU)
)
out = s.forward(data)
print(f'\nout : \n{out}')
s

[Linear] Patching Linear with activation: <class 'darth.fn.activation.ReLU'>
[Linear] activation of type: <class 'type'>
[Linear] Adding submodule: activation of type ReLU
[Linear] Patching Linear with activation: <class 'darth.fn.activation.ReLU'>
[Linear] activation of type: <class 'type'>
[Linear] Adding submodule: activation of type ReLU
[Sequential] Adding submodule: layer_0 of type Linear
[Sequential] Adding submodule: layer_1 of type Linear
[Sequential] forward called
[Sequential] _forward called - should be overridden in subclass
[Linear] forward called
[Linear] _forward Linear called with input shape (4, 4)
[Linear] apply_activation called with activation: None
[Linear] forward called
[Linear] _forward Linear called with input shape (4, 4)
[Linear] apply_activation called with activation: None
[Sequential] apply_activation called with activation: None

out : 
[[1.71758252 0.45397139]
 [1.47997749 0.79148062]
 [3.0822686  1.64986304]
 [1.27210666 0.79276442]]


Sequential(
  layer_0 = Linear(
    fan_in = 4,
    fan_out = 4,
    activation = ReLU(),
  ),
  layer_1 = Linear(
    fan_in = 4,
    fan_out = 2,
    activation = ReLU(),
  ),
)

In [ ]:
r.backward(1.2)

array([[0. , 0. , 1.2, 0. ],
       [1.2, 0. , 0. , 0. ],
       [1.2, 0. , 0. , 1.2],
       [1.2, 0. , 1.2, 0. ]])

# 🐛 Circular Dependency Analysis

Let's analyze and fix the circular imports in darth framework.

## Current Issues:
1. `darth.nn.layers` imports `Activation` from `darth.fn` (which doesn't exist)
2. `darth.fn.activation` imports `Module` from `darth.core`
3. Potential circular reference when Module tries to use activations

In [ ]:
# Let's test the current imports to see the exact error
try:
    print("Testing imports...")
    from darth.nn import Linear
    print("✅ Linear imported successfully")
    
    from darth.fn import av  
    print("✅ av imported successfully")
    
    # Try creating a Linear layer
    layer = Linear(4, 2, activation=av.sigmoid)
    print("✅ Linear layer created successfully")
    
except Exception as e:
    print(f"❌ Import error: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()